In [1]:
import pandas as pd
import numpy as np

In [8]:
scores = pd.read_csv('data_safe_copy.csv')
print(f'Row count: {len(scores)}')
print(scores.head())

Row count: 300
  student_id cohort   module assignment  score
0       S001  alpha  Module1         A1     78
1       S001  alpha  Module1         A2     84
2       S001  alpha  Module2         A1     79
3       S001  alpha  Module2         A2     86
4       S001  alpha  Module3         A1     81


In [9]:
scores.info()

<class 'pandas.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   student_id  300 non-null    str  
 1   cohort      300 non-null    str  
 2   module      300 non-null    str  
 3   assignment  300 non-null    str  
 4   score       300 non-null    int64
dtypes: int64(1), str(4)
memory usage: 11.8 KB


In [15]:
existing_ids = scores['student_id'].unique()[:6]

In [16]:
new_ids = [f'S{i}' for i in range(600, 610)]
overlap = scores[scores['student_id'].isin(new_ids)]
print(f'Is there any overlap for student ids: {len(overlap)}')

Is there any overlap for student ids: 0


In [17]:
all_roster_ids = np.concatenate([existing_ids, new_ids])

In [18]:
roster = pd.DataFrame({
    'student_id' : all_roster_ids,
    'status' : np.random.choice(['active', 'inactive'], size=len(all_roster_ids))
})

In [19]:
merged_df = pd.merge(scores, roster, on='student_id', how='left')

In [37]:
missing_count = merged_df['status'].isna().sum()

print(f"Records with missing status (NaN): {missing_count}")

Records with missing status (NaN): 264


In [38]:
avg_by_module = scores.groupby(['cohort', 'module'])['score'].mean().reset_index()

avg_by_cohort = scores.groupby('cohort')['score'].mean()

In [39]:
print(f'Average by moduls: \n{avg_by_module}')

Average by moduls: 
  cohort   module      score
0  alpha  Module1  77.970588
1  alpha  Module2  78.382353
2  alpha  Module3  79.794118
3   beta  Module1  78.647059
4   beta  Module2  78.411765
5   beta  Module3  80.029412
6  gamma  Module1  76.218750
7  gamma  Module2  76.468750
8  gamma  Module3  77.843750


In [40]:
print(f'Average by cohort: \n{avg_by_cohort}')

Average by cohort: 
cohort
alpha    78.715686
beta     79.029412
gamma    76.843750
Name: score, dtype: float64


In [41]:
comparison = avg_by_module.groupby('cohort')['score'].mean()

In [42]:
for cohort in avg_by_cohort.index:
    print(f"Cohort {cohort}:")
    print(f"  Overall Average: {avg_by_cohort[cohort]:.2f}")
    print(f"  Avg of Modules:  {comparison[cohort]:.2f}")

Cohort alpha:
  Overall Average: 78.72
  Avg of Modules:  78.72
Cohort beta:
  Overall Average: 79.03
  Avg of Modules:  79.03
Cohort gamma:
  Overall Average: 76.84
  Avg of Modules:  76.84


In [44]:
student_module_report = scores.pivot_table(
    index='student_id',
    columns='module',
    values='score',
    aggfunc='mean'
)

In [46]:
validation_for_student_ids = len(student_module_report) == scores['student_id'].nunique()
print(f"Student ids count match: {validation_for_student_ids}")

Student ids count match: True


In [52]:
print(f"Modules match: {list(student_module_report.columns) == sorted(scores['module'].unique())}")

Modules match: True


In [81]:
student_averages = scores.groupby(['student_id'])['score'].mean().reset_index()

In [82]:
student_cohort_map = scores[['student_id', 'cohort']].drop_duplicates()

In [83]:
student_averages_with_cohorts = pd.merge(student_averages, student_cohort_map, on='student_id')

In [84]:
top_students = (
    student_averages_with_cohorts.sort_values(['cohort', 'score'], ascending=[True, False])
    .groupby('cohort')
    .head(3)
)

In [90]:
top_students = top_students.rename(columns={'score': 'avg_score'})
top_students = top_students[['student_id', 'cohort', 'avg_score']]

In [91]:
cohort_counts = top_students['cohort'].value_counts()
cohort_counts

cohort
alpha    3
beta     3
gamma    3
Name: count, dtype: int64